In [1]:
import pyvisa
from qcodes.instrument_drivers.rohde_schwarz import (
    RohdeSchwarzZNB8,
    RohdeSchwarzZNBChannel,
)
import matplotlib.pyplot as plt
import matplotlib
%matplotlib qt

import numpy as np
from numpy import *
from scipy.fft import fft, ifft, fftshift
from scipy.io import savemat
from datetime import datetime
import time

import os
Global_data_adress = r"C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA"
exp_name = 'TWPA_noise'
adress = Global_data_adress+'\\'+exp_name
if not os.path.exists(adress):
  os.mkdir(adress)
  print("Folder %s created!" % adress)
else:
  print("Folder %s already exists" % adress)

Folder C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise already exists


In [2]:
now = datetime.now()
path = adress +'\\'+ now.strftime('%Y-%m-%d')
if not os.path.exists(path):
  os.mkdir(path)
  print("Folder %s created!" % path)
else:
  print("Folder %s already exists" % path)

Folder C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise\2023-09-26 already exists


# VNA init

In [3]:
def magtodb(mag): return 20*np.log10(mag)

def vna_setup_CW(cw_frequency=6e9, vna_power=-30, bandwidth=10):
    try: print(vna.query('*IDN?'))                 # check if connection already exist
    except:                                        # if no connection
        rm = pyvisa.ResourceManager()
        vna = rm.open_resource('GPIB0::20::INSTR') # GPIB connection
        print(vna.query('*IDN?'))
        
    vna.write(':SENS1:BAND {}'.format(str(bandwidth))) # set defoult bandwidth
    vna.write('SOUR1:POW {}'.format(str(vna_power)))   # set defoult power
    vna.write('INIT1:CONT OFF')                        # single-sweep mode
    vna.write('SENS1:SWE:POIN 201')                    # set maximum possible number of points for single sweep
    vna.write(':SENS1:SWE:TYPE CW')                    # set CW mode of measurements
    return vna

def vna_set_cw(): vna.write(':SENS1:SWE:TYPE CW')    

def vna_set_lin(): vna.write(':SENS1:SWE:TYPE LIN')    

def vna_setup_sweep(cw_frequency=6e9, vna_power=-30, bandwidth=10):
    try: print(vna.query('*IDN?'))                 # check if connection already exist
    except:                                        # if no connection
        rm = pyvisa.ResourceManager()
        vna = rm.open_resource('GPIB0::20::INSTR') # GPIB connection
        print(vna.query('*IDN?'))
        
    vna.write(':SENS1:BAND {}'.format(str(bandwidth))) # set defoult bandwidth
    vna.write('SOUR1:POW {}'.format(str(vna_power)))   # set defoult power
    vna.write('INIT1:CONT OFF')                        # single-sweep mode
    vna.write('SENS1:SWE:POIN 201')                    # set maximum possible number of points for single sweep
    vna.write(':SENS1:SWE:TYPE CW')                    # set CW mode of measurements
    return vna

def vna_on(): vna.write('OUTP ON')
def vna_off(): vna.write('OUTP OFF')
    
def vna_meas_points(): 
    return int(vna.query('SENS1:SWE:POIN?'))  # should be 201

def vna_sweep_time(): 
    return float(vna.query('SENS1:SWE:TIME?'))  # estimated sweep time for current settings of vna

def vna_set_cw_freq(freq): 
    vna.write('SENS1:FREQ:CW {}'.format(str(freq))) # set single frequency point for CW mode in Hz
    
def vna_set_sweep_freq(freq): 
    vna.write('SENS1:FREQ:START {}'.format(str(freq[0]))) 
    vna.write('SENS1:FREQ:STOP {}'.format(str(freq[-1]))) 
    vna.write('SENS1:SWE:POIN {}'.format(str(len(freq)))) 

def vna_set_power(power): 
    vna.write('SOUR1:POW {}'.format(str(power))) # set power
    
def vna_set_band(bandwidth): 
    vna.write(':SENS1:BAND {}'.format(str(bandwidth))) # set power
    
def vna_meas_cw():
    vna_on()
    vna.write("INIT1:IMM")                              # start measurement
    time.sleep(0.2+float(vna.query('SENS1:SWE:TIME?'))) # this part is important becouse the measurement nedds to finnish before readout process
    vna.write("CALC1:FORM MLOG")                        # format of data
    data_str = vna.query("CALC1:DATA? SDAT")            # readout
    
    data = np.array(data_str.rstrip().split(",")).astype("float64")
    s = data[0::2] + 1j * data[1::2]
    i = data[0::2]
    q = data[1::2]
    return abs(s), angle(s)

In [4]:
vna = vna_setup_CW() # initializing or checking for connection

Rohde-Schwarz,ZNB20-2Port,1311601062102129,3.45



# ANAPICO init

In [62]:
def setup_anapico(power=-30, frequency = 6e9, channel = 1):
    try: print(anapico.query('*IDN?'))
    except: 
        rm = pyvisa.ResourceManager()
#         anapico = rm.open_resource('USB0::0x03EB::0xAFFF::3C6-0B4F40003-0985::INSTR') # USB connection
        anapico = rm.open_resource('TCPIP0::169.254.5.91::inst0::INSTR') # LAN connection
        print(anapico.query('*IDN?'))
    if power<=-6:
#         print('Flux power set:', power)
        anapico.write(r'SOUR{}:POW {}'.format(str(channel), str(power)))
        anapico.write(r'SOUR{}:FREQ {}'.format(str(channel), str(frequency)))
    else: 
        print('Flux power is too high:', power)
    return anapico

def anapico_on(channel=1):
    anapico.write(r'OUTP{} ON'.format(str(channel)))

def anapico_off(channel=1):
    anapico.write(r'OUTP{} OFF'.format(str(channel)))

def anapico_freq(freq, channel=1): 
    anapico.write(r'SOUR{}:FREQ {}'.format(str(channel), str(freq)))

def anapico_power(power, channel=1): 
    anapico.write(r'SOUR{}:POW {}'.format(str(channel), str(power)))

In [107]:
anapico = setup_anapico()

AnaPico AG,APMS20G,3C6-0B4F40003-0985,0.4.204



In [105]:
anapico.close()

# ANRITSU MS2830A signal analizer init

In [55]:
def setup_anri():
    try: print(anri.query('*IDN?'))
    except: 
        rm = pyvisa.ResourceManager()
        anri = rm.open_resource('GPIB0::7::INSTR') # GPIB connection
        print(anri.query('*IDN?'))
    return anri


def anri_set_cent_freq(freq): 
    anri.write('FREQ:CENT {}'.format(str(freq)))
    
def anri_set_span(span): 
    anri.write('FREQ:SPAN {}'.format(str(span)))
    
def anri_set_band_kHz(band): 
    anri.write('BAND {}KHZ'.format(str(band)))

def anri_set_band_Hz(band): 
    anri.write('BAND {}HZ'.format(str(band)))
    
def anri_set_nop(nop):
    # max 10001
    anri.write('SWEep:POINts {}'.format(str(nop)))
    
    
def anri_get_nop():
    return int(anri.query('SWEep:POINts?'))

def anri_get_sweep_time():
    return float(anri.query('SWEep:TIME?'))

def sweep_mode_cont():
    anri.write('INIT:MODE:CONT')

def sweep_mode_sing():
    anri.write('INIT:MODE:SING')

def anri_get_data():
    anri.write('INIT:IMM')
    time.sleep(20+float(anri.query('SWEep:TIME?')))
    raw_data = anri.query('TRAC? TRAC1')
    data = np.array(raw_data.split(','), dtype = float)
    return data

In [8]:
anri = setup_anri()

ANRITSU,MS2830A,6200847105,4.09.01



In [9]:
anri_get_nop()

5001

In [108]:
anapico = setup_anapico()

flux_freq = 11.252e9
anapico_power(-6, 1)
anapico_freq(flux_freq, 1)
anapico_on(1)

mode1 = flux_freq/2+3e6
anapico_power(-6, 2)
anapico_freq(mode1, 2)
anapico_on(2)

mode2 = flux_freq/2+4e6
anapico_power(-6, 3)
anapico_freq(mode2, 3)
anapico_on(3)

time.sleep(5)
anapico.close()

band = 1
cent_freq = flux_freq/2
span = 45e6
nop = 10001
min_freq = cent_freq-span/2
max_freq = cent_freq+span/2
freq = np.linspace(min_freq, max_freq, nop)

# anri_set_band_kHz(band)
anri_set_nop(nop)
anri_set_band_Hz(band)
anri_set_cent_freq(cent_freq)
anri_set_span(span)

AnaPico AG,APMS20G,3C6-0B4F40003-0985,0.4.204



In [ ]:
data = anri_get_data()

fig, ax = plt.subplots(figsize=(15,7))
ax.set_ylim(-100, 0)
mp = cent_freq/1e9
m1 = mode1/1e9
m2 = mode2/1e9

ax.axvline(x=mp,  color = 'r', alpha = 0.7,label='$\omega_p/2$')
ax.axvline(x=m1, linestyle='--', alpha = 0.7, color = 'g', label='$\omega_1/2$')
ax.axvline(x=m2, linestyle='--', alpha = 0.7, color = 'b', label='$\omega_2/2$')

ax.axvline(x=2*mp-m1, linestyle='--', alpha = 0.7, color = 'g', label='2*mp-m1')
ax.axvline(x=2*mp-m2, linestyle='--', alpha = 0.7, color = 'b', label='2*mp-m2')


ax.axvline(x=m2+mp-m1, linestyle=':',  alpha = 0.7, color = 'orange',label='m2+mp-m1')
ax.axvline(x=-m2+mp+m1, linestyle=':',  alpha = 0.7, color = 'orange',label='-m2+mp+m1')
ax.axvline(x=m2-mp+m1, linestyle=':',  alpha = 1, color = 'yellow',label='m2-mp+m1')

ax.axvline(x=2*m1-mp, linestyle=':', alpha = 0.7, color = 'black', label='2*m1-mp')
ax.axvline(x=2*m2-mp, linestyle=':', alpha = 0.7,  color = 'aqua',label='2*m2-mp')
ax.axvline(x=2*m2-m1, linestyle=':', alpha = 0.7, color = 'greenyellow', label='2*m2-m1')
ax.axvline(x=2*m1-m2, linestyle=':', alpha = 0.7, color = 'deepskyblue', label='2*m1-m2')


ax.axvline(x=2*m1+m2-2*mp, linestyle=':', alpha = 0.7, color = 'violet', label='2*m1+m2-2*mp')
ax.axvline(x=2*m2+m1-2*mp, linestyle=':', alpha = 0.7, color = 'springgreen', label='2*m2+m1-2*mp')
ax.axvline(x=2*mp+m1-2*m2, linestyle=':', alpha = 0.7, color = 'violet', label='2*mp+m1-2*m2')
ax.axvline(x=2*mp+m2-2*m1, linestyle=':', alpha = 0.7, color = 'violet', label='2*mp+m2-2*m1')


ax.axvline(x=3*m1-2*mp, linestyle=':', alpha = 0.7, color = 'greenyellow', label='3*m1-2*mp')
ax.axvline(x=3*m2-2*mp, linestyle=':', alpha = 0.7, color = 'olivedrab', label='3*m2-2*mp')
ax.axvline(x=3*mp-2*m1, linestyle=':', alpha = 0.7, color = 'greenyellow', label='3*mp-2*m1')
ax.axvline(x=3*mp-2*m2, linestyle=':', alpha = 0.7, color = 'greenyellow', label='3*mp-2*m2')

ax.axvline(x=m1+2*m2-2*mp, linestyle=':', alpha = 0.7, color = 'springgreen', label='m1+2*m2-2*mp')
ax.axvline(x=m1+2*mp-2*m2, linestyle=':', alpha = 0.7, color = 'springgreen', label='m1+2*mp-2*m2')
ax.axvline(x=m2+2*m1-2*mp, linestyle=':', alpha = 0.7, color = 'springgreen', label='m2+2*m1-2*mp')
ax.axvline(x=m2+2*mp-2*m1, linestyle=':', alpha = 0.7, color = 'springgreen', label='m2+2*mp-2*m1')

ax.plot(freq[:-1]/1e9, data[:-1])

ax.legend()

In [103]:
def get_mode(i, j, k, mp, m1, m2):
    return i*mp+j*m1+k*m2

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

fig, ax = plt.subplots(figsize=(15,7))
ax.set_ylim(-100, 0)

mea = np.mean(data[:500])
std = np.std(data[:500])
ax.set_title('mean = {}, std = {}'.format(round(mea,2), round(std, 2)))
for i in np.array([-3, -2, -1, 0, 1, 2, 3]):
    for j in np.array([-3, -2, -1, 0, 1, 2, 3]):
        for k in np.array([-3, -2, -1, 0, 1, 2, 3]):
            mode = get_mode(i, j, k, mp, m1, m2)
            if mode<max(freq/1e9) and mode>min(freq/1e9):
                mode_ind = np.argwhere(freq/1e9==find_nearest(freq/1e9, mode))[0,0]
                mode_amp = max(data[mode_ind-10:mode_ind+10])
                
                if mode_amp>mea+std*3:
                    ax.axvline(mode, ymin=-100, ymax=1, linewidth=2, color = 'springgreen',
                               alpha = 1, label=r'${}\cdot m_p+{}\cdot m_1+{}\cdot m_2$'.format(i, j, k))

ax.plot(freq/1e9, data)
ax.legend()            